In [1]:
import requests
import lxml.html

import pandas as pd

import os

In [2]:
formData = {
    'model[Context]': 'atm',
    'model[ContextId]': '{FD212863-182A-4B0B-8132-AE5502D655CF}',
    'model[Page]': '1',
    'model[PageSize]': '5000',
    'model[Radius]': '5245',
    'model[Latitude]': '-6.19171113',
    'model[Longitude]': '106.8227999'
}

r = requests.post('https://www.bca.co.id/api/bca/lokasi/GetCard', data=formData)

tree = lxml.html.fromstring(r.text)

len(tree.xpath("//a[@class='show-route-maps']"))

5000

In [12]:
tree.xpath("//p[@class='a-text a-text-subtitle a-text-ellipsis-single m-maps-location-container-wrapper-title']")[0].text.strip()

'MRT BUNDARAN HOTEL INDONESIA 1'

In [11]:
tree.xpath("//p[@class='a-text a-text-body a-text-ellipsis-address m-maps-location-container-wrapper-address']")[0].text.strip()

'GONDANGDIA, KEC MENTENG, JAKARTA PUSAT'

In [16]:
len('https://www.google.com/maps/dir/?api=1&destination=')

51

In [15]:
tree.xpath("//a[@class='show-route-maps']")[0].attrib['href'][51:].split(',')

['-6.191577000', '106.823058000']

In [31]:
titles = [title.text.strip() for title in tree.xpath("//p[@class='a-text a-text-subtitle a-text-ellipsis-single m-maps-location-container-wrapper-title']")]
addresses = [address.text.strip() for address in tree.xpath("//p[@class='a-text a-text-body a-text-ellipsis-address m-maps-location-container-wrapper-address']")]
final_addresses = [f'{a} {b}' for a, b in zip(titles, addresses)]
lats = [coordinate.attrib['href'][51:].split(',')[0] for coordinate in tree.xpath("//a[@class='show-route-maps']")]
lngs = [coordinate.attrib['href'][51:].split(',')[1] for coordinate in tree.xpath("//a[@class='show-route-maps']")]

In [32]:
len(titles), len(addresses), len(final_addresses), len(lats), len(lngs)

(5000, 5000, 5000, 5000, 5000)

In [36]:
df = pd.DataFrame(data={'address': final_addresses, 'latitude': lats, 'longitude': lngs})
df['Bank'] = 'BCA'
df.to_csv('./bca/page1.csv', index=False)

# Automated

In [37]:
def get_lat_long(page: str):
  formData = {
      'model[Context]': 'atm',
      'model[ContextId]': '{FD212863-182A-4B0B-8132-AE5502D655CF}',
      'model[Page]': page,
      'model[PageSize]': '5000',
      'model[Radius]': '5245',
      'model[Latitude]': '-6.19171113',
      'model[Longitude]': '106.8227999'
  }

  r = requests.post('https://www.bca.co.id/api/bca/lokasi/GetCard', data=formData)

  tree = lxml.html.fromstring(r.text)

  titles = [title.text.strip() for title in tree.xpath("//p[@class='a-text a-text-subtitle a-text-ellipsis-single m-maps-location-container-wrapper-title']")]
  addresses = [address.text.strip() for address in tree.xpath("//p[@class='a-text a-text-body a-text-ellipsis-address m-maps-location-container-wrapper-address']")]
  final_addresses = [f'{a} {b}' for a, b in zip(titles, addresses)]
  lats = [coordinate.attrib['href'][51:].split(',')[0] for coordinate in tree.xpath("//a[@class='show-route-maps']")]
  lngs = [coordinate.attrib['href'][51:].split(',')[1] for coordinate in tree.xpath("//a[@class='show-route-maps']")]

  print(len(titles), len(addresses), len(final_addresses), len(lats), len(lngs))

  df = pd.DataFrame(data={'address': final_addresses, 'latitude': lats, 'longitude': lngs})
  df['Bank'] = 'BCA'
  df.to_csv(f'./bca/page{page}.csv', index=False)

In [41]:
get_lat_long(4)

1166 1166 1166 1166 1166


# Combined

In [4]:
df = pd.concat([pd.read_csv(f'./bca/{filepath}') for filepath in os.listdir('./bca/')])
df.to_csv('./bca/final.csv', index=False)